## Machine Learning with Timeseries Data

### Feature Engineering
- High amount of correlation in timeseries data. Need to conduct feature engineering on samples. 
- Some examples include:
 - Rolling Windows: Will give more information from data (mean, max and std) by smoothing data and making it positive. Specially used with audio data. 
   - For regressions, can use percentiles over each columns, date-based features etc. 
 - Spectogram: a collection of windowed **Fourier transforms** over time. Specially used in ***audio*** data. 
   - Choose a window size and shape.
   - At a timepoint, calculate FFT for that window
   - aggregate results
   - called **Short-Time Fourier Transform (STFT)**
   - for spectogram, we square the values of STFT

In [ ]:
'''Calculating Rolling Window Smoothing'''
#Calculating a rolling window statistic
#Smooth our data by taking the rolling mean in a window of 50 samples
window_size = 50
windowed = audio.rolling(window=window_size)
audio_smooth = windowed.mean()

#Two steps include:
#1 - Rectification
audio_rectified = audio.apply(np.abs)

#2 - Smoothing
audio_envelope = audio_rectified.rolling(50).mean()


#For regression data:

def percent_change(series):
    # Collect all *but* the last value of this window, then the final value
    previous_values = series[:-1]
    last_value = series[-1]

    # Calculate the % difference between the last value and the mean of earlier values
    percent_change = (last_value - np.mean(previous_values)) / np.mean(previous_values)
    return percent_change

# Apply your custom function and plot
prices_perc = prices.rolling(20).aggregate(percent_change)

# Define a rolling window with Pandas, excluding the right-most datapoint of the window
prices_perc_rolling = prices_perc.rolling(20, min_periods=5, closed='right')

# Define the features you'll calculate for each window
features_to_calculate = [np.min, np.max, np.mean, np.std]

# Calculate these features for your rolling window object
features = prices_perc_rolling.agg(features_to_calculate)

# Import partial from functools
from functools import partial
percentiles = [1, 10, 25, 50, 75, 90, 99]

# Use a list comprehension to create a partial function for each quantile
percentile_functions = [partial(np.percentile, q=percentile) for percentile in percentiles]

# Calculate each of these quantiles on the data using a rolling window
prices_perc_rolling = prices_perc.rolling(20, min_periods=5, closed='right')
features_percentiles = prices_perc_rolling.agg(percentile_functions)

''' STFT '''
# Import the stft function
import librosa as lr
from librosa.core import stft, amplitude_to_db
from librosa.display import specshow

# Prepare the STFT
HOP_LENGTH = 2**4
spec = stft(audio, hop_length=HOP_LENGTH, n_fft=2**7)

# Convert into decibels
spec_db = amplitude_to_db(spec)

# Calculate the spectral centroid and bandwidth for the spectrogram
bandwidths = lr.feature.spectral_bandwidth(S=spec)[0]
centroids = lr.feature.spectral_centroid(S=spec)[0]



## Auto-correlation in Timeseries Data
- Many times, the past data is a good predictor of future. More correlation leads to more smooth data
- The time period used for making rolling windows should be chosen carefully. If too wide, the data points will have highamount of correlation and will not be i.i.d. 
- We can use time shifted models to see how smooth or correlation lies in the data. 
- Use df.shift(periods = x) where x is number of indices. 
- If the coefficients slowly go to zero as shift increases, the data might not be i.i.d. the more erratic, the better!

Here are some steps you can take to address autocorrelation in your time series regression model:

- **Differencing**: One of the most common ways to handle autocorrelation is to difference the time series data. This involves subtracting the previous value from the current value. This can help make the data stationary, which is a key assumption for regression. You may need to do this multiple times until the data is stationary.

- **Lagged Variables**: You can include lagged values of the dependent variable or other relevant variables in your regression model. For example, if you are predicting a stock price, you can include lagged stock prices as predictors. This can help account for the autocorrelation in the data.

- **Autoregressive Models**: Consider using autoregressive models, such as ARIMA (AutoRegressive Integrated Moving Average) or SARIMA (Seasonal ARIMA), which are specifically designed for handling autocorrelation in time series data. These models can be used alongside regression when appropriate.

- **Moving Averages**: Another approach is to use moving averages to smooth the data and remove some of the noise. This can help in making the autocorrelation less pronounced.

- Time Series Cross-Validation: When training and evaluating your regression model, be sure to use time series cross-validation techniques, such as time-based splitting, to account for the temporal structure of the data.

- Residual Analysis: After building your model, analyze the residuals for autocorrelation. If there is still autocorrelation in the residuals, you may need to iterate and improve your model further.

- **Advanced Models**: Consider more advanced time series modeling techniques like state-space models or Bayesian structural time series **(BSTS) models**, which can handle complex temporal dependencies.

In [ ]:
#Example:
# These are the "time lags"
shifts = np.arange(1, 11).astype(int)

# Use a dictionary comprehension to create name: value pairs, one pair per shift
shifted_data = {"lag_{}_day".format(day_shift): prices_perc.shift(day_shift) for day_shift in shifts}

# Convert into a DataFrame for subsequent use
prices_perc_shifted = pd.DataFrame(shifted_data)

# Replace missing values with the median for each column
X = prices_perc_shifted.fillna(np.nanmedian(prices_perc_shifted))
y = prices_perc.fillna(np.nanmedian(prices_perc))

# Fit the model
model = Ridge()
model.fit(X, y)

## Cross Validation with Time Series Data
- Cannot just shuffle data since this will destroy the "temporal" nature of the data
- Shuffling will result in some data from training set seeping into test set. So the metric will be incorrect.
- **TimeSeriesSplit:** Special split function made for time series data for CV


In [ ]:
# Import TimeSeriesSplit
from sklearn.model_selection import TimeSeriesSplit

# Create time-series cross-validation object
cv = TimeSeriesSplit(n_splits=10)

# Iterate through CV splits
fig, ax = plt.subplots()
for ii, (tr, tt) in enumerate(cv.split(X, y)):
    # Plot the training data on each iteration, to see the behavior of the CV
    ax.plot(tr, ii + y[tr])

ax.set(title='Training data on each CV iteration', ylabel='CV iteration')
plt.show()

### Stability
- Change in relationship between X and y over time.
- How to check for? Bootstrapping and looking at confidence intervals and checking cross val scores

In [ ]:
from sklearn.utils import resample

def bootstrap_interval(data, percentiles=(2.5, 97.5), n_boots=100):
    """Bootstrap a confidence interval for the mean of columns of a 2-D dataset."""
    # Create our empty array to fill the results
    bootstrap_means = np.zeros([n_boots, data.shape[-1]])
    for ii in range(n_boots):
        # Generate random indices for our data *with* replacement, then take the sample mean
        random_sample = resample(data)
        bootstrap_means[ii] = random_sample.mean(axis=0)
        
    # Compute the percentiles of choice for the bootstrapped means
    percentiles = np.percentile(bootstrap_means, percentiles, axis=0)
    return percentiles



# Iterate through CV splits
n_splits = 100
cv = TimeSeriesSplit(n_splits=n_splits)

# Create empty array to collect coefficients
coefficients = np.zeros([n_splits, X.shape[1]])

for ii, (tr, tt) in enumerate(cv.split(X, y)):
    # Fit the model on training data and collect the coefficients
    model.fit(X[tr], y[tr])
    coefficients[ii] = model.coef_
    
# Calculate a confidence interval around each coefficient
bootstrapped_interval = bootstrap_interval(coefficients)

# Plot it
fig, ax = plt.subplots()
ax.scatter(feature_names, bootstrapped_interval[0], marker='_', lw=3)
ax.scatter(feature_names, bootstrapped_interval[1], marker='_', lw=3)
ax.set(title='95% confidence interval for model coefficients')
plt.setp(ax.get_xticklabels(), rotation=45, horizontalalignment='right')
plt.show()